In [64]:
#Requisitos do problema:
#Utilizando os arquivos de dowload na pasta atividades com os arquivos: Clientes, ItensVendas, Produtos, Vendas, Vendedores

#1. Crie uma consultaque mostre nesta ordem, Nome, Estados e Status dos clientes
#2. Crie uma consulta que mostre apenas os clientes do status platinum e gold
#3. Demonstre quanto cada status de clientes representa em vendas

#Resolva com dataframes, faça joins se necessario

In [65]:
#Instalando as bibliotecas:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col

In [66]:
#Criando o objeto spark:
spark = SparkSession.builder.appName("dataFrameUdemy").getOrCreate()

In [67]:
#Importanto arquivo clientes:
cliente = spark.read.format("parquet").load("/content/drive/MyDrive/download/Atividades/Clientes.parquet")

In [68]:
#Importando arquivo vendas:
venda = spark.read.format("parquet").load("/content/drive/MyDrive/download/Atividades/Vendas.parquet")

In [69]:
#Importando arquivo itensVendas:
itensVenda = spark.read.format("parquet").load("/content/drive/MyDrive/download/Atividades/ItensVendas.parquet")

In [70]:
cliente.show()

+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
|        6|       Adérito Bahía|    MA|     M|  Silver|
|        7|       Aida Dorneles|    RN|     F|  Silver|
|        8|   Alarico Quinterno|    AC|     M|  Silver|
|        9|    Alberto Cezimbra|    AM|     M|  Silver|
|       10|    Alberto Monsanto|    RN|     M|    Gold|
|       11|       Albino Canela|    AC|     M|  Silver|
|       12|     Alceste Varanda|    RR|     F|  Silver|
|       13|  Alcides Carvalhais|    RO|     M|  Silver|
|       14|        Aldo Martins|    GO|     M|  Silver|
|       15|   Alexandra Tabares|    MG|     F|  

In [71]:
#1. Crie uma consultaque mostre nesta ordem, Nome, Estados e Status dos clientes:
cliente1 = cliente.select("Cliente", "Estado", "Status").orderBy("Cliente")

In [72]:
cliente1.show(10)

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
+--------------------+------+--------+
only showing top 10 rows



In [73]:
cliente2 = cliente.select("ClienteID", "Cliente", "Estado", "Genero", "Status").where((f.col("Status") == "Platinum") | (f.col("Status") == "Gold")).orderBy("Cliente")

In [74]:
#2. Crie uma consulta que mostre apenas os clientes do status platinum e gold:
cliente.select("Cliente", "Status") \
    .where((f.col("Status") == "Platinum") | (f.col("Status") == "Gold")) \
    .orderBy("Status") \
    .show(20)

+-------------------+--------+
|            Cliente|  Status|
+-------------------+--------+
|   Alberto Monsanto|    Gold|
|      Anna Carvajal|    Gold|
|      Bento Quintão|    Gold|
|      Carminda Dias|    Gold|
|      Cláudio Jorge|    Gold|
|    Dionísio Saltão|    Gold|
|   Firmino Meireles|    Gold|
|Honorina Villaverde|    Gold|
|  Iracema Rodríguez|    Gold|
|   Adriana Guedelha|Platinum|
|      Flor Vilanova|Platinum|
|    Ibijara Botelho|Platinum|
|         Joana Ataí|Platinum|
+-------------------+--------+



In [75]:
# Fazendo o Join entre as tabelas cliente e vendas conforme MER
novoDataFrame1 = cliente.join(vendas, on="ClienteID", how="inner")


In [76]:
novoDataFrame1.show(5)

+---------+-----------------+------+------+------+--------+----------+--------+--------+
|ClienteID|          Cliente|Estado|Genero|Status|VendasID|VendedorID|    Data|   Total|
+---------+-----------------+------+------+------+--------+----------+--------+--------+
|       91|   Cosme Zambujal|    BA|     M|Silver|       1|         1|1/1/2019|  8053.6|
|      185|Gertrudes Hidalgo|    PA|     F|Silver|       2|         6|1/1/2020|   150.4|
|       31| Antão Corte-Real|    ES|     M|Silver|       3|         7|2/1/2020|  6087.0|
|       31| Antão Corte-Real|    ES|     M|Silver|       4|         5|2/1/2019| 13828.6|
|       31| Antão Corte-Real|    ES|     M|Silver|       5|         5|3/1/2018|26096.66|
+---------+-----------------+------+------+------+--------+----------+--------+--------+
only showing top 5 rows



In [77]:
## Fazendo o Join entre as tabelas novoDataFrame2 e itensVenda conforme MER
novoDataFrame2 = novoDataFrame1.join(itensVenda, on="VendasID", how="inner")

In [78]:
novoDataFrame2.show(5)

+--------+---------+-----------------+------+------+------+----------+--------+------+---------+----------+-------------+----------+--------+----------------+
|VendasID|ClienteID|          Cliente|Estado|Genero|Status|VendedorID|    Data| Total|ProdutoID|Quantidade|ValorUnitario|ValorTotal|Desconto|TotalComDesconto|
+--------+---------+-----------------+------+------+------+----------+--------+------+---------+----------+-------------+----------+--------+----------------+
|       1|       91|   Cosme Zambujal|    BA|     M|Silver|         1|1/1/2019|8053.6|        2|         1|      7820.85|   7820.85|     -  |         7820.85|
|       1|       91|   Cosme Zambujal|    BA|     M|Silver|         1|1/1/2019|8053.6|        9|         1|        97.75|     97.75|    0,98|           96.77|
|       1|       91|   Cosme Zambujal|    BA|     M|Silver|         1|1/1/2019|8053.6|       10|         1|        135.0|     135.0|    1,35|          133.65|
|       2|      185|Gertrudes Hidalgo|    PA| 

In [79]:
#3. Somando o valor das vendas de cada cliente.
dfTotal = novoDataFrame2.groupBy("ClienteID", "status").agg(f.sum("TotalComDesconto").cast("float").alias("SomaTotalComDesconto"))


In [80]:
dfTotal.show(5)

+---------+--------+--------------------+
|ClienteID|  status|SomaTotalComDesconto|
+---------+--------+--------------------+
|      233|  Silver|             76934.3|
|      230|Platinum|             9467.31|
|      241|  Silver|             5529.41|
|       31|  Silver|            580097.2|
|       46|  Silver|               97.75|
+---------+--------+--------------------+
only showing top 5 rows



In [81]:
#3. Demonstre quanto cada status de clientes representa em vendas
dfFinal = dfTotal.groupBy("status")\
.agg(f.sum("SomaTotalComDesconto")\
     .cast("float").alias("Total por categoria"))\
     .show()

+--------+-------------------+
|  status|Total por categoria|
+--------+-------------------+
|Platinum|            9467.31|
|  Silver|          2673780.0|
|    Gold|           21479.62|
+--------+-------------------+

